In [1]:
import numpy as np

from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=2000)

It took 00 mins and 04 seconds to download the candles


In [3]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    max_equity_risk_pct=np.array([4]) / 100,
    long_or_short=np.array([LongOrShortType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    num_candles=np.array([50, 150, 250]),
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [14]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.RegularBacktest,
    candles=candles,
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 6
Total combinations of settings to test: 24

Total candles: 2,000
Total candles to test: 48,000


In [15]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.CandleBacktest,
    candles=candles,
)
candle_backtest = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 6
Total combinations of settings to test: 24

Total candles: 2,000
Total candles to test: 48,000


In [39]:
candle_backtest.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
23,0,0,67.0,-72.389443,19.40,-0.948561,-419.9179,580.081927
20,0,1,68.0,-72.586750,19.12,-0.936491,-420.5812,579.418756
21,0,2,65.0,-70.315973,18.46,-0.944081,-412.8559,587.143991
8,0,3,60.0,-63.591409,15.00,-0.845967,-388.7210,611.279042
10,0,4,61.0,-71.317419,14.75,-0.860278,-416.2883,583.711804
11,0,5,59.0,-61.393553,15.25,-0.865053,-380.3966,619.603437
16,1,0,78.0,-127.040913,19.23,-0.929180,-559.5505,440.449250
18,1,1,78.0,-126.451942,19.23,-0.929926,-558.4050,441.594801
14,1,2,75.0,-126.177258,18.67,-0.924616,-557.8686,442.131101
3,1,3,72.0,-92.679281,16.67,-0.823366,-481.0028,518.997163


In [40]:
candles_reg.sort_values(['ind_set_idx','or_set_idx'])

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
20,0,0,68.0,-72.463994,19.12,-0.939925,-420.1687,579.831173
18,0,1,68.0,-72.586750,19.12,-0.936491,-420.5812,579.418756
22,0,2,65.0,-70.315973,18.46,-0.944081,-412.8559,587.143991
6,0,3,61.0,-65.834202,14.75,-0.840486,-396.9881,603.011915
8,0,4,61.0,-71.317419,14.75,-0.860278,-416.2883,583.711804
10,0,5,59.0,-61.393553,15.25,-0.865053,-380.3966,619.603437
14,1,0,78.0,-127.093343,19.23,-0.929185,-559.6522,440.347562
16,1,1,78.0,-126.451942,19.23,-0.929926,-558.4050,441.594801
12,1,2,75.0,-126.177258,18.67,-0.924616,-557.8686,442.131101
2,1,3,72.0,-92.679281,16.67,-0.823366,-481.0028,518.997163
